# Multi-Head-Attention

Multi-Head Attention은 입력 시퀀스를 여러 시각에서 정보를 동시에 학습하고, 다양한 관계를 포착할 수 있도록 설계되어있는 Transformer 아키텍처의 핵심 구성요소이다.<br>
다음과 같은 방법으로 입력 텐서에 대해 Query, Key, Value를 만들게 된다.

$$
Q = XW_Q, \quad K = XW_K, \quad V = XW_V
$$


In [46]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

batch = 2
seq = 8
n_dim = 50
n_head = 2

x = torch.randn(batch, seq, n_dim)

batch, seq, n_dim = x.size()
linear = nn.Linear(n_dim, n_dim * 3)
k, v, q = linear(x).split(n_dim, dim=-1)
k = k.view(batch, seq, n_head, n_dim // n_head).transpose(1, 2)
v = v.view(batch, seq, n_head, n_dim // n_head).transpose(1, 2)
q = q.view(batch, seq, n_head, n_dim // n_head).transpose(1, 2)

각 헤드별로 다음과 같은 Attention 연산을 수행하게 된다.

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$


`F.scaled_dot_product_attention` 함수를 사용하면 위의 연산을 간단하게 수행할 수 있다.

이때 매개변수 `is_casual`은 causal attention (또는 masked attention) 여부를 설정하는 데 사용한다.<br>
causal attention(masked attention)이란 현재 위치의 정보가 미래의 위치에 영향을 미치지 않도록 마스킹을 수행하는 매커니즘이다.

학습과 추론 모두 masked attention을 수행하므로 `is_casual`은 일반적으로 True로 세팅된다.


In [47]:
import torch.nn.functional as F

a = F.scaled_dot_product_attention(q,k,v, is_causal=True)
b = F.scaled_dot_product_attention(q,k,v, is_causal=False)

print(a.shape, b.shape)
print(torch.allclose(a,b))

torch.Size([2, 2, 8, 25]) torch.Size([2, 2, 8, 25])
False


# Masked Attention 

`F.scaled_dot_product_attention`로 간단하게 사용할 수 있지만 실제 수행되는 순서를 확인하기 위해 코드로 구현하겠다.

Query와 Key가 곱해지면 (seq_len, seq_len)의 matrix가 만들어지게 된다. (배치는 없다고 가정)<br>
이 행렬은 각 시퀀스간의 유사도를 의미하게 되는데, i행 j열의 경우 i번째 토큰과 j번째 토큰과의 유사도를 의미하게 된다.<br>
이때 i가 j보다 작은 경우 자신보다 뒤에 있는 단어(토큰 혹은 시퀀스)와 유사도를 계산하는 것인데, 실제 추론시에는 문장 전체를 알 수 없기 떄문에 이러한 경우는 문제가 발생한다.<br>
이러한 아직 생성되지 않은 단어에 대해 계산을 하는 경우를 방지하기 위해 마스킹을 수행한다.<br>

따라서 정방행렬의 대각선을 기준으로 upper triangular matrix을 0으로 만드는 작업을 마스킹이라고 한다. <br>
torch에서는 `torch.tril`을 통해 이를 수행한다.
tril은 triangular lower mask을 의미하며 입력 텐서의 lower triangular matrix를 1로 만든다.

In [48]:
max_seq = 8
mask = torch.tril(torch.ones(max_seq, max_seq)).view(1, 1, max_seq, max_seq)  # tril: triangular lower mask
 
print(mask)

tensor([[[[1., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 0., 0., 0., 0.],
          [1., 1., 1., 1., 1., 0., 0., 0.],
          [1., 1., 1., 1., 1., 1., 0., 0.],
          [1., 1., 1., 1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1., 1., 1., 1.]]]])


우선 $\frac{QK^T}{\sqrt{d_k}}$의 수식을 계산한다.

In [49]:
att = (q @ k.transpose(-2, -1)) / math.sqrt(k.size(-1))

이후 `mask`와 `torch.masked_fill`함수를 사용하여 마스킹을 수행한다.<br>
우삼각 부분을 -Inf 으로 만들면, Softmax시에 0으로 수렴하게 된다. (이 때문에 마스킹이라고 한다.)

`mask[:,:,:seq,:seq]`으로 최대 시퀀스 길이에 맞는 마스크 행렬을 만든다.<br>
0 값인 부분에 해당하는 원소를 `float('-inf')`로 만든다.

In [66]:
att = att.masked_fill(mask[:,:,:seq,:seq] == 0, float('-inf'))
att = F.softmax(att, dim=-1)
y = att @ v
print(y.shape)

torch.Size([2, 2, 8, 25])


최종 attention 결과가 아닌 $\text{softmax}(\frac{QK^T}{\sqrt{d_k}})$ 결과만 확인하면 아래와 같이 대각선의 윗부분이 0으로 마스킹 됨을 확인할 수 있다.

In [56]:
att[0][0] # 첫번째 배치의 첫번째 헤드

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5102, 0.4898, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3540, 0.3252, 0.3208, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2368, 0.2872, 0.2340, 0.2421, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2167, 0.1846, 0.2010, 0.2001, 0.1976, 0.0000, 0.0000, 0.0000],
        [0.1693, 0.1819, 0.1593, 0.1581, 0.1671, 0.1642, 0.0000, 0.0000],
        [0.1394, 0.1422, 0.1436, 0.1349, 0.1507, 0.1394, 0.1498, 0.0000],
        [0.1232, 0.1243, 0.1295, 0.1237, 0.1408, 0.1194, 0.1184, 0.1206]],
       grad_fn=<SelectBackward0>)

위의 연산을 통해 메모리의 연속성이 깨지므로 `contiguous`를 사용하고, <br>
`view` 함수를 통해 입력-출력의 크기를 동일하게 바꿔주면 Multi-Head-Attention 연산이 수행된다.

In [67]:
y = y.transpose(1, 2).contiguous().view(batch, seq, n_dim)

# 입력 - 출력 크기
print(x.shape==y.shape)

True
